# Namespace analysis

In [3]:
import os
from collections import Iterable, defaultdict
import warnings
import pandas as pd
import operator

from pybel import from_pickle, union
from pybel_tools import utils
from pybel.struct.mutation import remove_biological_processes, remove_filtered_nodes, collapse_to_genes
from pybel.struct.filters.node_predicate_builders import function_inclusion_filter_builder

from pathme.constants import REACTOME_BEL, KEGG_BEL, WIKIPATHWAYS_BEL
from pathme.utils import get_files_in_folder

from bio2bel_kegg import Manager as KeggManager
from bio2bel_reactome import Manager as ReactomeManager
from bio2bel_wikipathways import Manager as WikipathwaysManager

In [4]:
%matplotlib inline

In [5]:
# Initiate WikiPathways Manager
wikipathways_manager = WikipathwaysManager()
# Initiate Reactome Manager
reactome_manager = ReactomeManager()
# Initiate KEGG Manager
kegg_manager = KeggManager()

In [6]:
# Equivalent pathway IDs (ordered)
reactome_ids = ['R-HSA-5358508','R-HSA-209968','R-HSA-195721','R-HSA-5683057','R-HSA-71336','R-HSA-1257604','R-HSA-168898','R-HSA-983705','R-HSA-157118','R-HSA-109581','R-HSA-428157','R-HSA-5358351','R-HSA-71403','R-HSA-69306','R-HSA-5693571','R-HSA-1640170','R-HSA-9006936','R-HSA-165159','R-HSA-448424','R-HSA-74182','R-HSA-1170546']               
kegg_ids = ['hsa03430','hsa04918','hsa04310','hsa04010','hsa00030','hsa04151','hsa04620','hsa04662','hsa04330','hsa04210','hsa00600','hsa04340','hsa00020','hsa03030','hsa03450','hsa04110','hsa04350','hsa04150','hsa04657','hsa00072','hsa04917']
wikipathways_ids = ['WP531','WP1981','WP363','WP382','WP134','WP4172','WP75','WP23','WP61','WP254','WP1422','WP47','WP78','WP466','WP438','WP179','WP366','WP1471','WP2112','WP311','WP2037']

# Reactome pathways not contained in Reactome RDF file
REACTOME_BLACK_LIST = ['R-HSA-2025928','R-HSA-9604323', 'R-HSA-9013700','R-HSA-9017802','R-HSA-168927', 'R-HSA-9014325', 'R-HSA-9013508', 'R-HSA-9013973', 'R-HSA-9013957', 'R-HSA-9013695']

Methods used in this notebook

In [10]:
def flatten(l):
    for el in l:
        if isinstance(el, Iterable) and not isinstance(el, (str, bytes)):
            yield from flatten(el)
        else:
            yield el
            

def get_all_pathway_children_by_id(manager, reactome_id):
    
    pathway = manager.get_pathway_by_id(reactome_id)

    if not pathway.children:
        return pathway.reactome_id
    
    children = []
    
    for child in pathway.children:

        children.append(get_all_pathway_children_by_id(manager, child.reactome_id))
    
    return children


def get_bel_graph(resource_name, pathway_id):
    
    if resource_name == 'reactome':
        
        pickle_path = os.path.join(REACTOME_BEL, pathway_id + '.pickle')
        
    elif resource_name == 'kegg':
        
        pickle_path = os.path.join(KEGG_BEL, pathway_id + '_unflatten.pickle')
 
    elif resource_name == 'wikipathways':
        
        pickle_path = os.path.join(WIKIPATHWAYS_BEL, pathway_id + '.pickle')
    
    # Get BEL graph from pickle 
    bel_graph = from_pickle(pickle_path)
    
    return bel_graph


Create a dictionary for Reactome pathways with children

In [11]:
parent_to_child = dict()

for reactome_id in reactome_ids:
    
    all_children = get_all_pathway_children_by_id(reactome_manager, reactome_id)

    if isinstance(all_children, str):
        continue
        
    flattened_children = flatten(all_children)
    parent_to_child[reactome_id] = [pathway for pathway in flattened_children]

In [38]:
def get_namespaces_in_graph(graph):
    
    namespace_count = defaultdict(int)

    for node in graph:

        if 'namespace' in node.keys():

            namespace_count[node['namespace']] += 1

    return namespace_count

Get the merged network for every equivalent pathway

In [39]:
namespaces = dict()

for counter, reactome_id in enumerate(reactome_ids):
    
    reactome_graphs = []

    kegg_bel_graph = get_bel_graph('kegg', kegg_ids[counter])

    wikipathways_bel_graph = get_bel_graph('wikipathways', wikipathways_ids[counter])

    pathway_name = wikipathways_manager.get_pathway_by_id(wikipathways_ids[counter]) 
        
    # Check if reactome ID is in black list 
    if reactome_id in REACTOME_BLACK_LIST:
        continue
    
    # If Reactome parent pathway has children, get merged graph of children
    if reactome_id in parent_to_child:            
            
        pathway_children = parent_to_child[reactome_id]

        for child in pathway_children:
            if child not in REACTOME_BLACK_LIST:

                reactome_bel_graph = get_bel_graph('reactome', child)
                reactome_graphs.append(reactome_bel_graph)
                
    # Get Reactome parent pathway graph
    else:
        reactome_graphs.append(get_bel_graph('reactome', reactome_id))
    
    reactome_bel_graph = union(reactome_graphs)
    
    collapse_to_genes(kegg_bel_graph)
    collapse_to_genes(wikipathways_bel_graph)
    collapse_to_genes(reactome_bel_graph)

    namespaces["{}-{}".format(pathway_name, 'kegg')] = get_namespaces_in_graph(kegg_bel_graph)
    namespaces["{}-{}".format(pathway_name, 'wikipathways')] = get_namespaces_in_graph(wikipathways_bel_graph)
    namespaces["{}-{}".format(pathway_name, 'reactome')] = get_namespaces_in_graph(reactome_bel_graph)
    


In [41]:
for pathway, namespace in namespaces.items():
    
    print("{}: {}".format(pathway, namespace))

Mismatch repair-kegg: defaultdict(<class 'int'>, {'kegg': 2, 'HGNC': 23})
Mismatch repair-wikipathways: defaultdict(<class 'int'>, {})
Mismatch repair-reactome: defaultdict(<class 'int'>, {'61': 25, 'obo': 8, 'HGNC': 10})
Thyroxine (Thyroid Hormone) Production-kegg: defaultdict(<class 'int'>, {'ChEBI': 21, 'kegg': 7, 'HGNC': 74})
Thyroxine (Thyroid Hormone) Production-wikipathways: defaultdict(<class 'int'>, {'HGNC': 5, 'Interaction': 1, 'hmdb': 1})
Thyroxine (Thyroid Hormone) Production-reactome: defaultdict(<class 'int'>, {'obo': 6})
Wnt Signaling Pathway-kegg: defaultdict(<class 'int'>, {'ChEBI': 1, 'kegg': 9, 'HGNC': 149})
Wnt Signaling Pathway-wikipathways: defaultdict(<class 'int'>, {'HGNC': 34})
Wnt Signaling Pathway-reactome: defaultdict(<class 'int'>, {'61': 155, 'obo': 19, 'HGNC': 68, 'REACTOME': 25, 'ENSEMBL': 1, 'UniProt': 1})
MAPK Signaling Pathway-kegg: defaultdict(<class 'int'>, {'ChEBI': 5, 'kegg': 7, 'HGNC': 327})
MAPK Signaling Pathway-wikipathways: defaultdict(<class